# 2. Feature Generation

This module has the following purpose:

1. For every company, and for the quarters from 2009Q1 to 2018Q4, identify the active CEO and CFO in that quarter.

## 2.1 CEO/CFO identification
  
WikiData is not robust enough for this task. In fact, of the 5 first companies in the companies list, only 1 had info on their current CEO. None of the first 5 companies had any information on past CEOs, or any CFO info.

CEO and CFO are required to sign a legal certification as part of the filing. These forms are known as EXHIBIT 31.1 and EXHIBIT 31.2. This form always starts with: `I, Steven Roth, certify that:`  
Steven Roth is either CEO or CFO. Further down in this section the officer does sign with name and title, thus yielding CEO or CFO.  

These exhibits can be accessed directly, without the need to obtain and parse the full statement.
These exhibits are available on a 'Filing Detail' page with a structured URL.  
eg. https://www.sec.gov/Archives/edgar/data/**18498/000001849818000048/0000018498-18-000048**-index.htm
Note the structured portion of the url is made up of CIK and ADSH.

Summarized:
1. Get the list of adsh filing codes.
2. Access the URL that yields Filing Detail page.
3. Use BeautifulSoup to identify the URLs to EXHIBIT 31.1 and 31.2.
3. Use Regex and/or text matching to identify the officers.
4. Use Regex and/or text matching to identify their titles.

This will yield:  

| CIK        | Quarter           | CEO  | CFO |
| :------------ |:-------------:| -----:|-----:|
| 00002354   | 2015Q1 | Jim Jones | Tim Bucks |
| 00002354   | 2015Q2      |   Jim Jones | Tim Bucks |
| 00002354 | 2015Q3      | Jane Jackson | Tim Bucks |



In [1]:
# to work with data
import pandas as pd

# to work with regex
import re

# to download
import requests

# to work with HTML tags
from bs4 import BeautifulSoup

# to time functions
import datetime

# to use NaN
import numpy as np

# to pause
import time
import random

# to work with local files
import os

In [2]:
# restore the pickled dictionary of dataframes
try: 
    DataFrames
except NameError:
    DataFrames = pd.read_pickle('dict_of_dfs_num_pre_sub_tag.p')

In [3]:
# select the adsh and cik codes to build the URL
filings_to_obtain = DataFrames['SUB'][['adsh', 'cik']].copy()

In [4]:
def build_filing_detail_url(row):
    '''
    Returns url of format
    https://www.sec.gov/Archives/edgar/data/18498/000001849818000048/0000018498-18-000048-index.htm
    based on adsh and cik codes.
    '''
    adsh = str(row['adsh'])
    cik = str(row['cik'])
    adsh_stripped = adsh.replace('-','')
    
    url = 'https://www.sec.gov/Archives/edgar/data/' + cik + '/' + adsh_stripped + '/' + adsh + '-index.htm'
    return url

filings_to_obtain['URL'] = filings_to_obtain.apply(build_filing_detail_url, axis=1)

filings_to_obtain.head(3)

,adsh,cik,URL
0,0000002178-18-000067,2178,https://www.sec.gov/Archives/edgar/data/2178/0...
1,0000002488-18-000189,2488,https://www.sec.gov/Archives/edgar/data/2488/0...
2,0000002969-18-000044,2969,https://www.sec.gov/Archives/edgar/data/2969/0...


In [5]:
def clean(text):
    return BeautifulSoup(text).get_text()

def get_filings(URL):
    # Polite web scraping
    time.sleep(random.randint(1,3))
    
    r = requests.get(URL)
    soup = BeautifulSoup(r.text)
    
    exhibits_dict = {}
    
    for a in soup.find_all('a', href=True):
        # find like 'exhibit31.htm', 'fooexh31bar.html'
        if re.search(r'.*?ex[a-z]{0,5}31.*?.htm.?', str(a)):
            # The hrefs are relative to the domain: add domain.
            exhibit_url = 'https://www.sec.gov'+a['href']
            exhibit_title = a.string
            
            r_ex = requests.get(exhibit_url)
            exh_text = clean(r_ex.text)
            
            exhibits_dict[exhibit_title] = exh_text
    
    if exhibits_dict:
        return exhibits_dict
    else:
        return np.nan

if not 'filings_exh.csv' in os.listdir('.'):
    filings_to_obtain['Exhibits'] = filings_to_obtain['URL'].apply(get_filings)
    filings_to_obtain.to_csv('filings_exh.csv')
else:
    filings_to_obtain = pd.read_csv('filings_exh.csv', index_col=0)

In [6]:
filings_to_obtain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6421 entries, 0 to 6420
Data columns (total 4 columns):
adsh        6421 non-null object
cik         6421 non-null int64
URL         6421 non-null object
Exhibits    5527 non-null object
dtypes: int64(1), object(3)
memory usage: 250.8+ KB


In [7]:
filings_to_obtain.head(3)

,adsh,cik,URL,Exhibits
0,0000002178-18-000067,2178,https://www.sec.gov/Archives/edgar/data/2178/0...,{'a3q2018exhibit311.htm': '\nEX-31.1\n3\na3q20...
1,0000002488-18-000189,2488,https://www.sec.gov/Archives/edgar/data/2488/0...,{'exh311302certofceo-q318.htm': '\nEX-31.1\n3\...
2,0000002969-18-000044,2969,https://www.sec.gov/Archives/edgar/data/2969/0...,{'apd-exhibit311x30sep20.htm': '\nEX-31.1\n8\n...


In [8]:
def extract_names_titles(exh):
    regex = r'I,.(.+?),?.certify.that:.+?(Chief.+?Officer|(?<!Vice )President(?! and Chief))'
    exh = clean(str(exh))
    match = re.findall(regex, str(exh), re.DOTALL)
    if match:
        return match
    else:
        return np.nan

filings_to_obtain['Officers'] = filings_to_obtain['Exhibits'].apply(extract_names_titles)

In [9]:
# The names and titles are embedded in a list of tuples. 
# This cell structures them into two proper columns.

def return_name_given_title(officer_list, list_of_titles):
    if officer_list and type(officer_list) is list:
        result = []
        for tuples in officer_list:
            if tuples[1] in list_of_titles:
                if not tuples[0] in result:
                    result.append(tuples[0])
        if result:
            res_str = str(result[0])
            commapos = res_str.find(',')
            return res_str[:commapos]
        else:
            return np.nan
        
filings_to_obtain['CEO'] = filings_to_obtain['Officers'].apply(return_name_given_title, args=(
                                                                     ['Chief Executive Officer', 'President'], ))
filings_to_obtain['CFO'] = filings_to_obtain['Officers'].apply(return_name_given_title, args=(
                                                                     ['Chief Financial Officer'], ))

In [14]:
# Combine the results with the submission information ('SUB') dataframe.

CompInfo = DataFrames['SUB'][['adsh', 'cik', 'name', 'stprba', 'period', 'fy', 'fp', 'filed']].copy()
Company_CEO_CFO = pd.merge(
    CompInfo, 
    filings_to_obtain[['adsh', 'CEO', 'CFO']], 
    left_on='adsh', 
    right_on='adsh', 
    how='left')

In [16]:
# ca. 75% of filings have CEO and 62% have CFO.
Company_CEO_CFO.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6421 entries, 0 to 6420
Data columns (total 10 columns):
adsh      6421 non-null object
cik       6421 non-null int64
name      6421 non-null object
stprba    5875 non-null object
period    6421 non-null int64
fy        6420 non-null float64
fp        6419 non-null object
filed     6421 non-null int64
CEO       4724 non-null object
CFO       3958 non-null object
dtypes: float64(1), int64(3), object(6)
memory usage: 551.8+ KB


In [12]:
Company_CEO_CFO.head(3)

,adsh,cik,name,stprba,period,fy,fp,filed,CEO,CFO
0,0000002178-18-000067,2178,"ADAMS RESOURCES & ENERGY, INC.",TX,20180930,2018.0,Q3,20181107,NaN,Townes G. Pressle
1,0000002488-18-000189,2488,ADVANCED MICRO DEVICES INC,CA,20180930,2018.0,Q3,20181031,Lisa T. S,Devinder Kuma
2,0000002969-18-000044,2969,AIR PRODUCTS & CHEMICALS INC /DE/,PA,20180930,2018.0,FY,20181120,Seifi Ghasem,M. Scott Crocc


In [13]:
if not 'Company_CEO_CFO.csv' in os.listdir('.'):
    Company_CEO_CFO.to_csv('Company_CEO_CFO.csv')

## Result

We now have a CSV of CEO (74% of filings) and CFO (62% of filings) on filing date.